# Ashley Self Project: Wrangling and Analyzing Twitter Data

In [1]:
#Importing all libraries needed
import re
import json
import numpy as np
import pandas as pd
import requests
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth', 800)

np.random.seed(42)

### Data Gathering

Below are all three pieces of data needed to complete this project

1. Download the WeRateDogs Twitter archived data (twitter-archive-enhanced.csv)

In [ ]:
#Importing the twitter archived data provided by Udacity into a DataFrame
df_archive = pd.read_csv('twitter-archive-enhanced.csv')
df_archive.head()

In [ ]:
#Checking initial shape
df_archive.shape

2. Use the requests library to download the tweet image prediction (image_predictions.tsv)

In [ ]:
#Downloading the twitter image predictions tsv from Udacity
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
response.status_code    

In [5]:
#Saving the tsv locally
with open('image_predictions.tsv', mode='wb') as file:
    file.write(response.content)

In [6]:
#Importing the tsv into a DataFrame
df_images_predictions = pd.read_csv('image_predictions.tsv', sep='\t')

In [ ]:
#Checking first 5 rows for completion of prior code
df_images_predictions.head()

In [ ]:
#Checking dataframe shape
df_images_predictions.shape

3. Use the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

In [ ]:
!pip install -U tweepy==4.0

In [10]:
#Installing new library and fetching tweets from Twitter API
#I did use my API Key and Key Secret when running the code,
#but for security purposes are not displayed here
import tweepy

consumer_key = 'API Key.....' #left blank for security purposes
consumer_secret ='API Key Secret.....' #left blank for security purposes
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
api

In [ ]:
#Creating another DataFrame for Tweet IDs
tweet_ids = df_archive.tweet_id.values
len(tweet_ids)

In [ ]:
%%time

count = 0
fails_dict = {}

with open('tweet_json.txt', 'w') as outfile:
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            json.dump(tweet._json, outfile)
            print("Success")
            outfile.write('\n')
        except Exception as e:
            print("Fail")
            fails_dict[tweet_id] = e
        

print(fails_dict)

In [ ]:
#Checking length of new list
len(fails_dict)

In [ ]:
#Create new DataFrame from the tweet_json.txt file
json_df = pd.read_json('tweet_json.txt', lines=True, encoding='utf-8')
json_df = json_df.rename(columns={'id': 'tweet_id'})
json_df.head()

In [17]:
#Pulling in just these four columns
json_df = json_df[['tweet_id', 'favorite_count', 'retweet_count']]

In [ ]:
json_df.head()

### Assessing the Data

1. Assess the Twitter Archive file

In [ ]:
#First, assess the twitter archive file visually,
#looking for quality and tidiness
#A summary of findings will be included later
pd.options.display.max_colwidth = 10000
df_archive

In [20]:
df_archive['time_stamp'] = pd.to_datetime(df_archive['timestamp'])

In [ ]:
#Assess the data programmatically
df_archive.info()

We know that most of the columns with NaN are common, as most tweets in this dataset are not replies and/or retweets, however, we should still work to drop all retweets to clean up the data.

In [ ]:
#Look at expanded_urls to see if NaN values are retweets or original tweets
df_archive[df_archive['expanded_urls'].isnull()].loc[:, ['expanded_urls',
                                                         'in_reply_to_status_id',
                                                         'in_reply_to_user_id',
                                                         'retweeted_status_id']]

We can see in the code above that only four expanded_urls with NaN are original tweets, and not replies/retweets (185, 375, 707, 1445).

In [ ]:
#By expanding these four tweets, we see that the first one (185) is a retweet by the RT@ in the text column - issue found!
url_nan_original_tweets = df_archive.iloc[[185, 375, 707, 1445], :].tweet_id
df_archive.iloc[[185, 375, 707, 1445], :]

#### Quality and Tidiness Issues - df_archive

##### Quality
1. Too many retweets present in the dataframe, we want original tweets only
2. The text column also contains text followed by a url, this should be cleaned up and either separated, or condensed to just text
3. Need to change data type from int to string for tweet_id column

##### Tidiness
1. The last four columns: doggo, floofer, pupper, puppo can be combined into one column with a filter to condense and clean up the data
2. Spitting the timestamp column would be helpful - separate them into two columns: date and time

2. Assess the Image Predictions file

In [ ]:
#First, assess the image predictions file visually,
#looking for quality and tidiness
#A summary of findings will be included later
df_images_predictions

In [ ]:
#Assess the data programmatically
df_images_predictions.info()

Through programmatic assessing in the code above, we see that no columns have NaN values!

In [ ]:
df_images_predictions.describe()

In [ ]:
df_images_predictions.sample(10)

There is no data that seems out of place with the descriptive statistics of this df or the sample above.

In [ ]:
sum(df_images_predictions.duplicated(subset=['jpg_url']))

#### Quality and Tidiness Issues - df_images_predictions

##### Quality

1. There is a mix of capitalized and non-capitalized dog breeds in multiple columns (needs consistency)
2. Multiple columns have non-descriptive names (ie: p1, p1_conf, p1_dog, p2, p2_conf, p2_dog, p3, p3_conf, p3_dog)
3. Need to drop the 66 duplicates in the jpg_url column

3. Assess the Tweet json file

In [ ]:
#First, assess the image predictions file visually,
#looking for quality and tidiness
#A summary of findings will be included later
json_df

In [ ]:
#Assess the data programmatically
json_df.info()

Through programmatic assessing in the code above, we see that no columns have NaN values, too!

#### Quality and Tidiness Issues - json_df

##### Quality

1. Too many retweets present in the dataframe, we want original tweets only

### Cleaning the Data

In [31]:
#Make a copy of all three datasets
df_archive_clean = df_archive.copy()
df_images_predictions_clean = df_images_predictions.copy()
json_df_clean = json_df.copy()

###### Address Quality Issue:
1. & 2. Too many retweets present in the dataframe, we want original tweets only (with images) - addresses two quality issues
            Cleans both NaN values and deletes unecessary columns with retweeted info and replies

In [32]:
df_archive_clean = df_archive_clean[pd.isnull(df_archive_clean['retweeted_status_id'])]

In [ ]:
#Testing code to make sure it worked
print(sum(df_archive_clean.retweeted_status_id.value_counts()))

In [ ]:
df_archive_clean.info()

Filtered/deleted all retweeted information through above code - works by showing retweeted columns as 0 counted.

In [35]:
#Get rid of columns related to retweets and replies
df_archive_clean = df_archive_clean.drop(['retweeted_status_id',
                                          'retweeted_status_user_id',
                                          'retweeted_status_timestamp',
                                          'in_reply_to_status_id',
                                          'in_reply_to_user_id'],
                                         axis=1)

In [ ]:
#Make sure it worked
df_archive_clean.info()

Works - issues resolved!

###### Address Quality Issue:

3. The text column also contains text followed by a url, this should be cleaned up and either separated, or condensed to just text

In [37]:
#Cleaned output will be stored in a list
def new_list(dataframe, column, word):
    text_list = []
    for text in dataframe[column]:
        text_list.append(text)
    clean_text_list = []
    for text_2 in text_list:
        clean_text = text_2[:text_2.find(word)-1]
        clean_text_list.append(clean_text)
    return(clean_text_list)

In [38]:
#Create a new column with the clean text list to make sure the code works
df_archive_clean['new_text'] = new_list(df_archive_clean, 'text', 'https')

In [ ]:
#View old and new column to make sure all code works
df_archive_clean.loc[:, ['text', 'new_text']]

It worked! Now lets input the new column into the old column, overriding the original data

In [40]:
df_archive_clean['text'] = new_list(df_archive_clean, 'text', 'https')
df_archive_clean = df_archive_clean.drop(['new_text'], axis=1)

In [ ]:
#Check to make sure code works
df_archive_clean.head()

Viola - it worked! Issue resolved.

###### Address Quality Issue:

4. Need to change data type from int to string for tweet_id column in all files

In [42]:
df_archive_clean['tweet_id'] = df_archive_clean['tweet_id'].astype('str')
df_images_predictions_clean['tweet_id'] = df_images_predictions_clean['tweet_id'].astype('str')
json_df_clean['tweet_id'] = json_df_clean['tweet_id'].astype('str')

In [ ]:
#Check to make sure code works
df_archive_clean.info()
df_images_predictions_clean.info()
json_df_clean.info()

Works - issue resolved!

###### Address Quality Issue:

5. After looking at the data programmatically, we find that some of the urls in expanded_urls are missing
            *We can look at tweet_id to possibly get the missing urls

In [44]:
def add_urls(row):
    if pd.notnull(row['expanded_urls']):
        return row
    else:
        tweet_id = row['tweet_id']
        row['expanded_urls'] = 'https://twitter.com/dog_rates/status/{}'.format(tweet_id)
        return row

df_archive_clean = df_archive_clean.apply(add_urls, axis=1)

In [ ]:
#Check to make sure code works
df_archive_clean[df_archive_clean['expanded_urls'].isnull()]
df_archive_clean.head()

###### Address Quality Issue:

6. There is a mix of capitalized and non-capitalized dog breeds in multiple columns (needs consistency)

In [47]:
df_images_predictions_clean['p1'] = df_images_predictions_clean['p1'].str.title()
df_images_predictions_clean['p2'] = df_images_predictions_clean['p2'].str.title()
df_images_predictions_clean['p3'] = df_images_predictions_clean['p3'].str.title()

In [ ]:
#Check to make sure code works
df_images_predictions_clean.loc[:,['p1', 'p2', 'p3']].head()

Works - issue resolved!

###### Address Quality Issue:

7. Multiple columns have non-descriptive names (ie: p1, p1_conf, p1_dog, p2, p2_conf, p2_dog, p3, p3_conf, p3_dog)

In [50]:
df_images_predictions_clean = df_images_predictions_clean.rename(columns={'jpg_url': 'image_url',
                                                                  'img_num': 'image_number',
                                                                 'p1': 'top_prediction',
                                                                  'p1_conf': 'top_prediction_confidence',
                                                                  'p2': 'second_prediction',
                                                                  'p2_conf': 'second_prediction_confidence',
                                                                  'p3': 'third_prediction',
                                                                  'p3_conf': 'third_prediction_confidence',
                                                                  'p1_dog': 'top_dog_prediction',
                                                                  'p2_dog': 'second_dog_prediction',
                                                                  'p3_dog': 'third_dog_prediction',
                                                                 })

In [ ]:
#Check to make sure code works
df_images_predictions_clean.head()

Works - issue resolved!

###### Address Quality Issue:

8. Need to drop the 66 duplicates in the jpg_url column

In [52]:
df_images_predictions_clean = df_images_predictions_clean.drop_duplicates(subset = ['image_url'], keep='last')

In [ ]:
#Check to make sure code works
sum(df_images_predictions_clean['image_url'].duplicated())

Works - issue resolved!

###### Address Tidiness Issue:

1. The last four columns: doggo, floofer, pupper, puppo can be combined into one column with a filter to condense and clean up the data

In [ ]:
df_archive_clean['doggo'].replace('None', '', inplace=True)
df_archive_clean['floofer'].replace('None', '', inplace=True)
df_archive_clean['pupper'].replace('None', '', inplace=True)
df_archive_clean['puppo'].replace('None', '', inplace=True)

In [ ]:
df_archive_clean['level_of_dog'] = (df_archive_clean['doggo'] +
                                    df_archive_clean['floofer'] +
                                    df_archive_clean['pupper'] +
                                    df_archive_clean['puppo'])

In [ ]:
df_archive_clean = df_archive_clean.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis=1)

In [71]:
df_archive_clean['level_of_dog'].replace('', 'NaN', inplace=True)

In [ ]:
#Check to make sure code works
df_archive_clean['level_of_dog'].value_counts()

In [73]:
df_archive_clean.loc[df_archive_clean.level_of_dog == 'doggopupper'] = 'doggo, pupper'
df_archive_clean.loc[df_archive_clean.level_of_dog == 'doggopuppo'] = 'doggo, puppo'
df_archive_clean.loc[df_archive_clean.level_of_dog == 'doggofloofer'] = 'doggo, floofer'

In [ ]:
df_archive_clean['level_of_dog'].value_counts()

Works - fixed Udacity Reviewers required change for level of dog.

In [ ]:
df_archive_clean.info()
df_archive_clean.tail()

Works - issue resolved!

###### Address Tidiness Issue:

2. Spitting the timestamp column would be helpful and make the data easier to read

In [ ]:
df_archive_clean['temp_time'] = pd.DatetimeIndex(df_archive_clean['timestamp']) 

In [199]:
#Splitting date, year, month
temp_time = pd.DatetimeIndex(df_archive_clean['timestamp'])

df_archive_clean['Date'] = temp_time.date
df_archive_clean['Year'] = temp_time.year
df_archive_clean['Month'] = temp_time.month

In [200]:
#Substituting month name for number
df_archive_clean['Month'].replace(1, 'January', inplace=True)
df_archive_clean['Month'].replace(2, 'February', inplace=True)
df_archive_clean['Month'].replace(3, 'March', inplace=True)
df_archive_clean['Month'].replace(4, 'April', inplace=True)
df_archive_clean['Month'].replace(5, 'May', inplace=True)
df_archive_clean['Month'].replace(6, 'June', inplace=True)
df_archive_clean['Month'].replace(7, 'July', inplace=True)
df_archive_clean['Month'].replace(8, 'August', inplace=True)
df_archive_clean['Month'].replace(9, 'September', inplace=True)
df_archive_clean['Month'].replace(10, 'October', inplace=True)
df_archive_clean['Month'].replace(11, 'November', inplace=True)
df_archive_clean['Month'].replace(12, 'December', inplace=True)

In [201]:
df_archive_clean['Day_of_Week'] = temp_time.dayofweek

In [202]:
#Substituting day of week name for number
df_archive_clean['Day_of_Week'].replace(0, 'Monday', inplace=True)
df_archive_clean['Day_of_Week'].replace(1, 'Tuesday', inplace=True)
df_archive_clean['Day_of_Week'].replace(2, 'Wednesday', inplace=True)
df_archive_clean['Day_of_Week'].replace(3, 'Thursday', inplace=True)
df_archive_clean['Day_of_Week'].replace(4, 'Friday', inplace=True)
df_archive_clean['Day_of_Week'].replace(5, 'Saturday', inplace=True)
df_archive_clean['Day_of_Week'].replace(6, 'Sunday', inplace=True)

In [203]:
df_archive_clean['Time'] = temp_time.time
df_archive_clean['Hour'] = temp_time.hour

df_archive_clean = df_archive_clean.drop('timestamp', axis=1)

In [ ]:
#Check to make sure code works
df_archive_clean.info()
df_archive_clean.head()

### Merge all tables into one dataset

In [ ]:
#Merge archive and image
twitter_1 = pd.merge(df_archive_clean, 
                     df_images_predictions_clean, 
                     how = 'left', on = 'tweet_id')

#keep rows that have image_url
twitter_1 = twitter_1[twitter_1['image_url'].notnull()]

#Check to make sure code works
twitter_1.info()
twitter_1.head()

In [ ]:
#Merge twitter_1 and json
twitter_2 = pd.merge(twitter_1, json_df_clean, 
                      how = 'left', on = 'tweet_id')

#Check to make sure code works
twitter_2.info()
twitter_2.head()

In [175]:
#Now we store the newly merged dataset in twitter_archive_master.csv
twitter_2.to_csv('twitter_archive_master.csv', 
                 index=False, encoding = 'utf-8')

In [ ]:
pd.read_csv('twitter_archive_master.csv')

### Analyze and Visualize

__First Insight__

What is the start and finish date of the data?

In [ ]:
print(twitter_2['Date'][-1:][len(twitter_2)-1])
print(twitter_2['Date'][0])

- The dataset contains data from __11-15-2015__ through __08-01-2017__.

__Second Insight__ - two questions

What is the most common dog?

In [ ]:
twitter_2['top_prediction'].value_counts()

__Golden Retriever__ is the most common dog in the dataset.

What are the top 5 most common dog names?

In [ ]:
twitter_2['name'].value_counts()[:7]

__Charlie__ is the most common dog name in the dataset.

__Third Insight__

Let's look at the descriptive statistics of the overall data

In [ ]:
twitter_2.describe()

- The __mean rating__ for this dataset is __12.3__.
- On average, __7,017__ twitter users __favorited__ each WeRateDogs original tweets.
- However, the __average retweet count__ is significantly lower at only __2,006__ retweets.
- The tweet with the __highest likes__ had __142,045__ likes.
- The __most retweeted tweet__ had __69,593__ retweets.
- The tweet with the __lowest amount of likes__ only had __64__ likes.
- The __least retweeted tweet__ only had __11__ retweets.

In [ ]:
twitter_2.groupby(['Month', 'Year'], 
                               sort=False).count().iloc[::-1, 0]

In [196]:
tweets_per_month = twitter_2.groupby(['Month', 'Year'], 
                               sort=False).count().iloc[::-1, 0]

In [ ]:
fig, ax = plt.subplots()
sns.set_style("white")
tweets_per_month.plot(kind='bar', color='b')
ax.set_title('Number of original tweets per month', fontsize=14, fontweight="bold")
ax.set_ylabel('count', fontsize=12)
ax.set_xlabel('')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

__November 2015__ and __December 2015__ were the busiest months for WeRateDogs twitter account with the number of original posted tweets, both exceeding 250. From __April 2016__ to __August 2017__, every month had less than 100 tweets total.

Some code has been adapted from:
- https://github.com/MrGeislinger/UdacityDAND_Proj_WrangleAndAnalyzeData
- https://github.com/latinacode/Wrangle-and-Analyze-Data
- https://github.com/NTavou/Wrangle_and_Analyze_Twitter_Data
- https://github.com/gouravaich/wrangle-analyze-weratedogs-twitter